# [xTuring](https://github.com/stochasticai/xturing) - LLaMA efficient fine-tuning tutorial

This tutorial aims to show how easy it is to perform fine-tuning with xTuring. This notebook shows how to fine-tune LLaMA 7B model on GPU which has limited memory, it requires only 5GB VRAM

## 1. Install the `xTuring` library

In [1]:
!pip install xturing --upgrade

     |████████████████████████████████| 86 kB 6.8 MB/s             
  Attempting uninstall: xturing
    Found existing installation: xturing 0.0.5
    Uninstalling xturing-0.0.5:
      Successfully uninstalled xturing-0.0.5
You should consider upgrading via the '/Users/romanageev/Documents/GitHub/turing-test/venv/bin/python -m pip install --upgrade pip' command.


## Load and run docker image

1. Install Docker on your machine if you haven't already. You can follow the [official Docker documentation](https://docs.docker.com/engine/install/) for installation instructions.
2. Run the Docker daemon
    ```bash
    sudo systemctl start docker
    ```


In [ ]:
from xturing.utils.docker import run_docker_container, pull_docker_image

image = "public.ecr.aws/t8g5g2q5/xturing:int4_finetuning"
port_mapping = "5000:5000"
env_vars = {
    "WANDB_MODE": "dryrun",
}
# if you want to log results to wandb, set the following env var
# env_vars = {
#     "WANDB_API_KEY": "<your_wandb_api_key>",
#     "WANDB_PROJECT": "your_project_name",
#     "WANDB_ENTITY": "your_entity_name",
#     # Add more environment variables as needed
# }
gpus = "all"

pull_docker_image(image)

run_docker_container(image, port_mapping, env_vars, gpus)

int4_finetuning: Pulling from t8g5g2q5/xturing
fb668870d8a7: Pulling fs layer
4542784317be: Pulling fs layer
e0bec5df5af5: Pulling fs layer
4053f75740ab: Pulling fs layer
57e09105cdfd: Pulling fs layer
606761d225e5: Pulling fs layer
69473a703fb4: Pulling fs layer
a08ab4e0594b: Pulling fs layer
4cd507bccac2: Pulling fs layer
fa92f16621a4: Pulling fs layer
6dc2b05bd224: Pulling fs layer
bdd5f6455bdd: Pulling fs layer
48d915a4e6f6: Pulling fs layer
2a70c506252e: Pulling fs layer
4f4fb700ef54: Pulling fs layer
bfe9228f18ac: Pulling fs layer
515667f151fa: Pulling fs layer
fe05a1fd255e: Pulling fs layer
483edfdf157b: Pulling fs layer
e633d788041b: Pulling fs layer
b9ea26c9d146: Pulling fs layer
26c397f269e8: Pulling fs layer
5ce30301b654: Pulling fs layer
26775b6d6a02: Pulling fs layer
bdd5f6455bdd: Waiting
48d915a4e6f6: Waiting
e633d788041b: Waiting
2a70c506252e: Waiting
b9ea26c9d146: Waiting
26c397f269e8: Waiting
4f4fb700ef54: Waiting
5ce30301b654: Waiting
26775b6d6a02: Waiting
bfe9228f18a